In [4]:
import time
import math
from pprint import pprint
from itertools import islice

import numpy as np
import torch
from torch import nn

import syft as sy
from syft import VirtualMachine
from syft.core.plan.plan_builder import make_plan, ROOT_CLIENT
from syft.lib.python.collections.ordered_dict import OrderedDict
from syft import logger

import transformers
from transformers.models.distilbert.modeling_distilbert import DistilBertConfig
from transformers import AutoConfig, AutoTokenizer, PreTrainedTokenizerFast
import datasets

In [5]:
# Create client
alice = sy.VirtualMachine(name="alice")
alice_client = alice.get_client()
remote_torch = ROOT_CLIENT.torch

## Load DistilBert

In [6]:
# Load model
model = transformers.AutoModel.from_pretrained("distilbert-base-uncased")
tokenizer = transformers.AutoTokenizer.from_pretrained("distilbert-base-uncased")
config = model.config

# Turn off dropout
model = model.eval()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
model_ptr = model.send(alice_client)

In [8]:
tokenizer_ptr = tokenizer.send(alice_client)

## Load some data

In [9]:
train_set = datasets.load_dataset('sst', split='train')
train_loader = torch.utils.data.DataLoader(train_set, batch_size=32, shuffle=True)

num_labels = len(set(train_set['label']))
config.num_labels = num_labels

bx = next(iter(train_loader))['sentence']
bx_ptr = sy.lib.python.List(bx).send(alice_client)

No config specified, defaulting to: sst/default


ModuleNotFoundError: No module named 'pyarrow.vendored'

Exception ignored in: 'pyarrow.lib._PandasAPIShim._have_pandas_internal'
Traceback (most recent call last):
  File "pyarrow/pandas-shim.pxi", line 110, in pyarrow.lib._PandasAPIShim._check_import
  File "pyarrow/pandas-shim.pxi", line 56, in pyarrow.lib._PandasAPIShim._import_pandas
ModuleNotFoundError: No module named 'pyarrow.vendored'


Dataset sst downloaded and prepared to /Users/atrask/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff. Subsequent calls will reuse this data.


In [10]:
# Tokenize on client
bt_ptr = tokenizer_ptr(bx_ptr, padding=True, return_tensors='pt', truncation=True)
print(bt_ptr)

In [11]:
# Forward on client
model_ptr.eval()
out_ptr = model_ptr(bt_ptr['input_ids'], bt_ptr['attention_mask'], return_dict=False)
client_out = out_ptr.get()

/Users/atrask/Laboratory/openmined/PySyft/packages/syft/src/syft/lib/torch/uppercase_tensor.py:30: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  grad = getattr(obj, "grad", None)


In [12]:
bt = tokenizer(bx, padding=True, return_tensors='pt', truncation=True)
local_out = model(bt['input_ids'], bt['attention_mask'], return_dict=False)

In [13]:
torch.mean(torch.abs(client_out[0] - local_out[0]))

tensor(0., grad_fn=<MeanBackward0>)